<a href="https://colab.research.google.com/github/ToumaTanaka/Data_Science/blob/main/Mathematical_Optimization/Allocation_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 割当問題
* 0-1整数計画問題として解く(解ベクトルxの各要素を0または1のみに限定したもの)
* PythonのソルバーであるPuLPを利用する
* 割り当てた時の目的関数が最小になる組み合わせを探す

In [1]:
!pip install pulp

     |████████████████████████████████| 41.2 MB 69 kB/s 


In [2]:
import pulp
import pandas as pd

In [3]:
#トラックを35台と出勤者を35人とした時のデータを用意
#scoreは資料の(出勤者の累積仕事量)×(トラックの仕事量)＋(出勤者の累積取上)×(トラックの取上)の仮想的な数値だとする

df = pd.read_csv('output.csv')
df.head()

,Unnamed: 0,track,driver,score
0,0,1,1,77
1,1,1,2,68
2,2,1,3,76
3,3,1,4,79
4,4,1,5,97


In [4]:
df['Var'] = [pulp.LpVariable(f'x{df.track[L]}_{df.driver[L]}',cat="Binary") for L in df.index]
df

,Unnamed: 0,track,driver,score,Var
0,0,1,1,77,x1_1
1,1,1,2,68,x1_2
2,2,1,3,76,x1_3
3,3,1,4,79,x1_4
4,4,1,5,97,x1_5
...,...,...,...,...,...
1220,1220,35,31,93,x35_31
1221,1221,35,32,97,x35_32
1222,1222,35,33,98,x35_33
1223,1223,35,34,61,x35_34


In [5]:
#最小値探索のモデルを定義
problem = pulp.LpProblem('割り当て問題', sense=pulp.LpMinimize)

#定義したモデルに制約式を追加
#トラックとドライバーの数が違う時ダミーの数値が同じ値をおく
#ドライバー1人にトラック一台を割り当てる制約
for j, v in df.groupby('driver'):
  problem += pulp.lpSum(v.Var) <= 1
#トラック一台に1人だけ割り当てる
for i, v in df.groupby('track'):
  problem += pulp.lpSum(v.Var) == 1

#定義したモデルに目的関数を追加
problem += pulp.lpDot(df.score,df.Var)

In [6]:
#解を求める
result = problem.solve()

#最適解が求められたか表示(最適解なら'optimal'と表示される)
print(pulp.LpStatus[result])

#目的関数の値を表示
print(pulp.value(problem.objective))

#最適化結果をデータフレームに追加
df['Val'] = df.Var.apply(pulp.value)

Optimal
1818.0


In [7]:
df.head()

,Unnamed: 0,track,driver,score,Var,Val
0,0,1,1,77,x1_1,0.0
1,1,1,2,68,x1_2,0.0
2,2,1,3,76,x1_3,0.0
3,3,1,4,79,x1_4,0.0
4,4,1,5,97,x1_5,0.0


In [8]:
df_s = df[df.Val > 0][['track', 'driver', 'score']]
df_s

,track,driver,score
11,1,12,50
54,2,20,50
95,3,26,50
138,4,34,50
170,5,31,52
184,6,10,58
244,7,35,50
261,8,17,54
281,9,2,52
328,10,14,54


In [ ]:
df_s.to_csv("optimal.csv")
files.download("optimal.csv") 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>